In [1]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
import os
import random

import cv2
import keras.utils
import numpy as np
from keras.layers import Input, Dense,Flatten, Concatenate
from keras.models import Model
from keras.ops import empty

keras.utils.set_random_seed(42)
np.random.seed(42)
random.seed(42)


###TRAINING VARIABLES

sample_count = 79999
epoch_count = 100
batch_size = 15
validation_count = 20000
initial_learning_rate=.001
decay_steps=8888
decay_rate=0.97
dataset_name = "SuToBoCM ResNetV2"



inp = Input(shape=(320, 240, 1))


resnet = keras.applications.ResNet50V2(
    include_top=False,
    weights=None,
    input_tensor=inp,
    input_shape=(320, 240, 1),
    pooling=max,
    classifier_activation="relu",
)

flat = Flatten()(resnet.output)
dense = Dense(128, activation="relu")(flat)

chest_out = Dense(1, activation='linear', name='chest_out')(dense)
waist_out = Dense(1, activation='linear', name='waist_out')(dense)
pelvis_out = Dense(1, activation='linear', name='pelvis_out')(dense)
bicep_out = Dense(1, activation='linear', name='bicep_out')(dense)
thigh_out = Dense(1, activation='linear', name='thigh_out')(dense)
shoulder_to_wrist_out = Dense(1, activation='linear', name='shoulder_to_wrist_out')(dense)
leg_out = Dense(1, activation='linear', name='leg_out')(dense)
calf_out = Dense(1, activation='linear', name='calf_out')(dense)
wrist_out = Dense(1, activation='linear', name='wrist_out')(dense)
shoulder_to_shoulder_out = Dense(1, activation='linear', name='shoulder_to_shoulder_out')(dense)


model = Model(
    inputs=inp,
    outputs=[
        chest_out,
        waist_out,
        pelvis_out,
        bicep_out,
        thigh_out,
        shoulder_to_wrist_out,
        leg_out,
        calf_out,
        wrist_out,
        shoulder_to_shoulder_out,
    ],
    name='ResNet50V2'
)

folder_path = f"{dataset_name} - {sample_count}s{epoch_count}e{validation_count}v {"" if initial_learning_rate == 0.001 else initial_learning_rate + "ilr"}{decay_steps}s{decay_rate}d"

try:
    os.mkdir(os.path.join("./logs/" + folder_path))
except OSError as error:
    print(error)
    

#model.save_weights("default.weights.h5")

# model.summary()
# 
# from keras.utils import plot_model
# 
# plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True, show_layer_activations=True)

In [2]:
class Database_Loader(keras.utils.Sequence):

    def __init__(self, image_location, data_location, samples, batch_size, dataset, shuffle=True, seed=0, input_dimensions=(240, 320), prefix="Avatar_", side=False, height = False):
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.seed = seed
        self.input_dimensions = input_dimensions
        self.prefix = prefix
        self.side = side
        self.height = height

        super().__init__(workers=2, use_multiprocessing=True)
        self.IDs = []
        self._load_data(samples, data_location, image_location, dataset)
        self.on_epoch_end()


    def __len__(self):
        return int(np.floor(len(self.IDs) / self.batch_size))


    def _load_data(self, samples, data_locations, image_locations, datasets):
        self.IDs = []
        self.data = dict()
        id_counter = 0
        for data_source_index in range(len(image_locations)):
            source_image_count = samples[data_source_index]
            source_data_location = data_locations[data_source_index]
            source_image_location = image_locations[data_source_index]
            source_dataset = datasets[data_source_index]

            for index in range(source_image_count):
                self.IDs.append(id_counter)
                self.data[id_counter] = [source_image_location,
                                         source_dataset,
                                         index,
                                         np.load(os.path.join(source_data_location + self.prefix + f"{index:06d}.npy"))]
                id_counter += 1

    def __getitem__(self, index):
        X = np.empty(shape=(self.batch_size, 320, 240))
        if self.side:
            X2 = np.empty(shape=(self.batch_size, 320, 240))
        y = {
            'chest_out': [],
            'waist_out': [],
            'pelvis_out': [],
            'bicep_out': [],
            'thigh_out': [],
            'shoulder_to_wrist_out': [],
            'leg_out': [],
            'calf_out': [],
            'wrist_out': [],
            'shoulder_to_shoulder_out': [],
        }

        start_index = index * self.batch_size + 1
        for i in range(self.batch_size):
            sample_ID = self.IDs[(start_index + i)]
            sample_properties = self.data[sample_ID]
            current_measurement = sample_properties[3]
            current_filename = f"{self.prefix}{sample_properties[2]:06d}.png"

            X[i,] = cv2.imread(os.path.join(sample_properties[0]+ "images_front/" + current_filename), cv2.IMREAD_GRAYSCALE)
            if self.side:
                X2[i,] = cv2.imread(os.path.join(sample_properties[0]+ "images_side/" + current_filename), cv2.IMREAD_GRAYSCALE)

            height = []

            if sample_properties[1] == "Surreact":
                y['chest_out'].append([current_measurement[0]])
                y['waist_out'].append([current_measurement[1]])
                y['pelvis_out'].append([current_measurement[2]])
                y['bicep_out'].append([current_measurement[4]])
                y['thigh_out'].append([current_measurement[5]])
                y['shoulder_to_wrist_out'].append([current_measurement[7]])
                y['leg_out'].append([current_measurement[8]])
                y['calf_out'].append([current_measurement[9]])
                y['wrist_out'].append([current_measurement[11]])
                y['shoulder_to_shoulder_out'].append([current_measurement[13]])
                if self.height:
                    height.append(current_measurement[16])

            if sample_properties[1] == "BodyM":
                y['chest_out'].append([current_measurement[4]])
                y['waist_out'].append([current_measurement[12]])
                y['pelvis_out'].append([current_measurement[7]])
                y['bicep_out'].append([current_measurement[2]])
                y['thigh_out'].append([current_measurement[11]])
                y['shoulder_to_wrist_out'].append([current_measurement[1]])
                y['leg_out'].append([current_measurement[8]])
                y['calf_out'].append([current_measurement[3]])
                y['wrist_out'].append([current_measurement[13]])
                y['shoulder_to_shoulder_out'].append([current_measurement[9]])
                if self.height:
                    height.append(current_measurement[6])


        for key, value in y.items():
            y[key] = np.array(value)

        if self.side and self.height:
            height = np.array(height)
            return [X,X2, height], y
        if self.height:
            height = np.array(height)
            return [X, height], y
        if self.side:
            return [X,X2], y

        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.IDs)
        else:
            self.IDs = np.arange(len(self.IDs))


In [3]:
quickTrain = {
                'image_location': "./Surreact-APose/train/images_front/", 
                'data_location': "./Surreact-APose/train/measurementsCM/", 
                'sample_count': sample_count, 
                'batch_size': batch_size, 
                'seed':  69,#np.random.randint(0, 10000), 
                'use_memory': False,
                'random_sample': False,
                'shuffle': True}

quickValidate = {
                'image_location': "./Surreact-APose/test/images_front/",
                'data_location': "./Surreact-APose/test/measurementsCM/",
                'sample_count': validation_count,
                'batch_size': batch_size,
                'seed': 69, #np.random.randint(0, 10000),
                'use_memory': False,
                'random_sample': False}

train_generator = Database_Loader(**quickTrain)
validation_generator = Database_Loader(**quickValidate)

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping
from keras.losses import MeanSquaredError


lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True)

stop_no_improvement = keras.callbacks.EarlyStopping(monitor="loss", patience=3)


model.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss={
        'chest_out': MeanSquaredError(),
        'waist_out': MeanSquaredError(),
        'pelvis_out': MeanSquaredError(),
        'bicep_out': MeanSquaredError(),
        'thigh_out': MeanSquaredError(),
        'shoulder_to_wrist_out': MeanSquaredError(),
        'leg_out': MeanSquaredError(),
        'calf_out': MeanSquaredError(),
        'wrist_out': MeanSquaredError(),
        'shoulder_to_shoulder_out': MeanSquaredError(),
    },
    metrics={
        'chest_out': ['mae'],
        'waist_out': ['mae'],
        'pelvis_out': ['mae'],
        'bicep_out': ['mae'],
        'thigh_out': ['mae'],
        'shoulder_to_wrist_out': ['mae'],
        'leg_out': ['mae'],
        'calf_out': ['mae'],
        'wrist_out': ['mae'],
        'shoulder_to_shoulder_out': ['mae'],
    },
)



checkpoint_filepath = './models/ResNet50V2Base.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='min',
    save_best_only=True)

model.fit(
    x=train_generator,
    validation_data=validation_generator,
    callbacks=[
        
        TensorBoard(write_graph=True,log_dir="./logs/"+folder_path),
        model_checkpoint_callback,
    ],
    batch_size=batch_size,
    epochs=epoch_count,
    verbose=1,
)

model.save("./models/full/ResNet50V2Base.keras")